# EBAC - Regressão II - regressão múltipla

## Tarefa I

#### Previsão de renda II

Vamos continuar trabalhando com a base 'previsao_de_renda.csv', que é a base do seu próximo projeto. Vamos usar os recursos que vimos até aqui nesta base.

|variavel|descrição|
|-|-|
|data_ref                | Data de referência de coleta das variáveis |
|index                   | Código de identificação do cliente|
|sexo                    | Sexo do cliente|
|posse_de_veiculo        | Indica se o cliente possui veículo|
|posse_de_imovel         | Indica se o cliente possui imóvel|
|qtd_filhos              | Quantidade de filhos do cliente|
|tipo_renda              | Tipo de renda do cliente|
|educacao                | Grau de instrução do cliente|
|estado_civil            | Estado civil do cliente|
|tipo_residencia         | Tipo de residência do cliente (própria, alugada etc)|
|idade                   | Idade do cliente|
|tempo_emprego           | Tempo no emprego atual|
|qt_pessoas_residencia   | Quantidade de pessoas que moram na residência|
|renda                   | Renda em reais|

In [1]:
import pandas as pd
import numpy as np

import statsmodels.formula.api as smf
import statsmodels.api as sm
import patsy

from sklearn.model_selection import train_test_split
from sklearn.linear_model import RidgeCV
from sklearn.linear_model import LassoCV
from sklearn.metrics import r2_score

In [2]:
df = pd.read_csv('previsao_de_renda.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             15000 non-null  int64  
 1   data_ref               15000 non-null  object 
 2   id_cliente             15000 non-null  int64  
 3   sexo                   15000 non-null  object 
 4   posse_de_veiculo       15000 non-null  bool   
 5   posse_de_imovel        15000 non-null  bool   
 6   qtd_filhos             15000 non-null  int64  
 7   tipo_renda             15000 non-null  object 
 8   educacao               15000 non-null  object 
 9   estado_civil           15000 non-null  object 
 10  tipo_residencia        15000 non-null  object 
 11  idade                  15000 non-null  int64  
 12  tempo_emprego          12427 non-null  float64
 13  qt_pessoas_residencia  15000 non-null  float64
 14  renda                  15000 non-null  float64
dtypes:

1. Separe a base em treinamento e teste (25% para teste, 75% para treinamento).
2. Rode uma regularização *ridge* com alpha = [0, 0.001, 0.005, 0.01, 0.05, 0.1] e avalie o $R^2$ na base de testes. Qual o melhor modelo?
3. Faça o mesmo que no passo 2, com uma regressão *LASSO*. Qual método chega a um melhor resultado?
4. Rode um modelo *stepwise*. Avalie o $R^2$ na vase de testes. Qual o melhor resultado?
5. Compare os parâmetros e avalie eventuais diferenças. Qual modelo você acha o melhor de todos?
6. Partindo dos modelos que você ajustou, tente melhorar o $R^2$ na base de testes. Use a criatividade, veja se consegue inserir alguma transformação ou combinação de variáveis.
7. Ajuste uma árvore de regressão e veja se consegue um $R^2$ melhor com ela.

In [4]:
df.drop(['Unnamed: 0', 'data_ref', 'id_cliente'], axis=1, inplace=True)
df.head()

,sexo,posse_de_veiculo,posse_de_imovel,qtd_filhos,tipo_renda,educacao,estado_civil,tipo_residencia,idade,tempo_emprego,qt_pessoas_residencia,renda
0,F,False,True,0,Empresário,Secundário,Solteiro,Casa,26,6.602740,1.0,8060.34
1,M,True,True,0,Assalariado,Superior completo,Casado,Casa,28,7.183562,2.0,1852.15
2,F,True,True,0,Empresário,Superior completo,Casado,Casa,35,0.838356,2.0,2253.89
3,F,False,True,1,Servidor público,Superior completo,Casado,Casa,30,4.846575,3.0,6600.77
4,M,True,False,0,Assalariado,Secundário,Solteiro,Governamental,33,4.293151,1.0,6475.97


In [5]:
df = pd.get_dummies(df)

In [6]:
df.isna().sum()

posse_de_veiculo                    0
posse_de_imovel                     0
qtd_filhos                          0
idade                               0
tempo_emprego                    2573
qt_pessoas_residencia               0
renda                               0
sexo_F                              0
sexo_M                              0
tipo_renda_Assalariado              0
tipo_renda_Bolsista                 0
tipo_renda_Empresário               0
tipo_renda_Pensionista              0
tipo_renda_Servidor público         0
educacao_Primário                   0
educacao_Pós graduação              0
educacao_Secundário                 0
educacao_Superior completo          0
educacao_Superior incompleto        0
estado_civil_Casado                 0
estado_civil_Separado               0
estado_civil_Solteiro               0
estado_civil_União                  0
estado_civil_Viúvo                  0
tipo_residencia_Aluguel             0
tipo_residencia_Casa                0
tipo_residen

In [7]:
df = df.dropna()
df.isna().sum()

posse_de_veiculo                 0
posse_de_imovel                  0
qtd_filhos                       0
idade                            0
tempo_emprego                    0
qt_pessoas_residencia            0
renda                            0
sexo_F                           0
sexo_M                           0
tipo_renda_Assalariado           0
tipo_renda_Bolsista              0
tipo_renda_Empresário            0
tipo_renda_Pensionista           0
tipo_renda_Servidor público      0
educacao_Primário                0
educacao_Pós graduação           0
educacao_Secundário              0
educacao_Superior completo       0
educacao_Superior incompleto     0
estado_civil_Casado              0
estado_civil_Separado            0
estado_civil_Solteiro            0
estado_civil_União               0
estado_civil_Viúvo               0
tipo_residencia_Aluguel          0
tipo_residencia_Casa             0
tipo_residencia_Com os pais      0
tipo_residencia_Comunitário      0
tipo_residencia_Estú

In [8]:
X = df.drop(columns=['renda'])
y = df['renda']

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [10]:
#######2
# o codigo a seguir foi desenvolvido a partir de pesquisas, pois nao foi mostrado como fazer a regularizacao ridge.

# alpha a serem testados
alpha_v = [0.00000001, 0.001, 0.005, 0.01, 0.05, 0.1]

# regressao ridge com cross-validation, para depois escolhermos o melhor valor de alpha
ridge_cv = RidgeCV(alphas=alpha_v, store_cv_values=True)
ridge_cv.fit(X_train, y_train)

# melhor alpha
b_alpha = ridge_cv.alpha_
print('melhor alpha=', b_alpha)

y_pred = ridge_cv.predict(X_test)
R2 = r2_score(y_test, y_pred)
print('R2=', R2)


melhor alpha= 0.1
R2= 0.23773617377744038


In [11]:
#######3
from warnings import filterwarnings
filterwarnings('ignore')

# regressao Lasso com cross-validation
lasso_cv = LassoCV(alphas=alpha_v, cv=5)
lasso_cv.fit(X_train, y_train)

b_alpha = ridge_cv.alpha_
print('melhor alpha=', b_alpha)

y_pred = lasso_cv.predict(X_test)
R2 = r2_score(y_test, y_pred)
print('R2=', R2)

melhor alpha= 0.1
R2= 0.23775105166280142


In [12]:
# a regressao lasso ficou com R2 sensivelmente melhor

In [13]:
#######4
def stepwise_selection(X, y, 
                       initial_list=[], 
                       threshold_in=0.05, 
                       threshold_out = 0.05, 
                       verbose=True):
    """ Perform a forward-backward feature selection 
    based on p-value from statsmodels.api.OLS
    Arguments:
        X - pandas.DataFrame with candidate features
        y - list-like with the target
        initial_list - list of features to start with (column names of X)
        threshold_in - include a feature if its p-value < threshold_in
        threshold_out - exclude a feature if its p-value > threshold_out
        verbose - whether to print the sequence of inclusions and exclusions
    Returns: list of selected features 
    Always set threshold_in < threshold_out to avoid infinite looping.
    See https://en.wikipedia.org/wiki/Stepwise_regression for the details
    """
    included = list(initial_list)
    while True:
        changed=False
        # forward step
        excluded = list(set(X.columns)-set(included))
        new_pval = pd.Series(index=excluded, dtype=np.dtype('float64'))
        for new_column in excluded:
            model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included+[new_column]]))).fit()
            new_pval[new_column] = model.pvalues[new_column]
        best_pval = new_pval.min()
        if best_pval < threshold_in:
            best_feature = new_pval.index[new_pval.argmin()]
            included.append(best_feature)
            changed=True
            if verbose:
                 print('Add  {:30} with p-value {:.6}'.format(best_feature, best_pval))

        # backward step
        print("#############")
        print(included)
        model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included]))).fit()
        # use all coefs except intercept
        pvalues = model.pvalues.iloc[1:]
        worst_pval = pvalues.max() # null if pvalues is empty
        if worst_pval > threshold_out:
            changed=True
            worst_feature = pvalues.argmax()
            included.remove(worst_feature)
            if verbose:
                print('Drop {:30} with p-value {:.6}'.format(worst_feature, worst_pval))
        if not changed:
            break
    return included

In [14]:
# a funcao stepwise nao suporta valores True e False
cols_conversao = ['posse_de_veiculo', 'posse_de_imovel', 'sexo_F', 'sexo_M',
       'tipo_renda_Assalariado', 'tipo_renda_Bolsista',
       'tipo_renda_Empresário', 'tipo_renda_Pensionista',
       'tipo_renda_Servidor público', 'educacao_Primário',
       'educacao_Pós graduação', 'educacao_Secundário',
       'educacao_Superior completo', 'educacao_Superior incompleto',
       'estado_civil_Casado', 'estado_civil_Separado', 'estado_civil_Solteiro',
       'estado_civil_União', 'estado_civil_Viúvo', 'tipo_residencia_Aluguel',
       'tipo_residencia_Casa', 'tipo_residencia_Com os pais',
       'tipo_residencia_Comunitário', 'tipo_residencia_Estúdio',
       'tipo_residencia_Governamental']

X_train = X_train[cols_conversao].astype(int)
X_test = X_test[cols_conversao].astype(int)

In [15]:
res = stepwise_selection(X_train, y_train)
print('Variaveis selecionadas\n', res)

Add  sexo_F                         with p-value 5.95081e-158
#############
['sexo_F']
Add  tipo_renda_Servidor público    with p-value 4.50981e-07
#############
['sexo_F', 'tipo_renda_Servidor público']
Add  sexo_M                         with p-value 0.0
#############
['sexo_F', 'tipo_renda_Servidor público', 'sexo_M']
Add  tipo_residencia_Com os pais    with p-value 6.22516e-06
#############
['sexo_F', 'tipo_renda_Servidor público', 'sexo_M', 'tipo_residencia_Com os pais']
Add  educacao_Superior incompleto   with p-value 0.000299446
#############
['sexo_F', 'tipo_renda_Servidor público', 'sexo_M', 'tipo_residencia_Com os pais', 'educacao_Superior incompleto']
Add  estado_civil_Casado            with p-value 0.0200642
#############
['sexo_F', 'tipo_renda_Servidor público', 'sexo_M', 'tipo_residencia_Com os pais', 'educacao_Superior incompleto', 'estado_civil_Casado']
Add  posse_de_imovel                with p-value 0.0242603
#############
['sexo_F', 'tipo_renda_Servidor público', 'se

In [16]:
reg_sw = sm.OLS(y_train, sm.add_constant(X_train[res])).fit()
reg_sw.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  renda   R-squared:                       0.082
Model:                            OLS   Adj. R-squared:                  0.081
Method:                 Least Squares   F-statistic:                     103.8
Date:                Sun, 30 Jun 2024   Prob (F-statistic):          2.00e-166
Time:                        17:26:06   Log-Likelihood:                -97633.
No. Observations:                9320   AIC:                         1.953e+05
Df Residuals:                    9311   BIC:                         1.953e+05
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
================================================================================================
                                   coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------
const                         3942.3122    163.587     24.099      0.000    3621.645    4262.979
sexo_F                        -610.9918    115.156     -5.306      0.000    -836.722    -385.261
tipo_renda_Servidor público   1429.4367    293.963      4.863      0.000     853.204    2005.669
sexo_M                        4553.3040    133.244     34.173      0.000    4292.117    4814.491
tipo_residencia_Com os pais  -1538.6206    394.660     -3.899      0.000   -2312.241    -765.000
educacao_Superior incompleto -1326.9500    439.750     -3.018      0.003   -2188.955    -464.945
estado_civil_Casado            639.6062    215.884      2.963      0.003     216.427    1062.785
posse_de_imovel                439.5062    189.574      2.318      0.020      67.900     811.113
estado_civil_Separado          934.1043    417.554      2.237      0.025     115.606    1752.602
educacao_Superior completo     386.3498    186.352      2.073      0.038      21.059     751.641
==============================================================================
Omnibus:                    12298.121   Durbin-Watson:                   1.988
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          3488810.999
Skew:                           7.341   Prob(JB):                         0.00
Kurtosis:                      96.640   Cond. No.                     8.94e+15
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 3.2e-28. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

In [17]:
y_pred = reg_sw.predict(sm.add_constant(X_test[res]))
R2 = r2_score(y_test, y_pred)
print('R2=', R2)

R2= 0.0635773608973087


In [18]:
#######5
# analisando o R2 eu manteria a ridge ou lasso, mais facil implementacao

In [19]:
#######6
# fazendo a regressao lasso no log da renda

y_train_log = np.log(y_train)
y_test_log = np.log(y_test)

In [20]:
lasso_cv = LassoCV(alphas=alpha_v, cv=5)
lasso_cv.fit(X_train, y_train_log)

b_alpha = ridge_cv.alpha_
print('melhor alpha=', b_alpha)

y_pred = lasso_cv.predict(X_test)
R2 = r2_score(y_test_log, y_pred)
print('R2=', R2)

melhor alpha= 0.1
R2= 0.13829909138322272


In [21]:
# fazendo a regressao ridge no log da renda
ridge_cv = RidgeCV(alphas=alpha_v, store_cv_values=True)
ridge_cv.fit(X_train, y_train_log)

b_alpha = ridge_cv.alpha_
print('melhor alpha=', b_alpha)

y_pred = ridge_cv.predict(X_test)
R2 = r2_score(y_test_log, y_pred)
print('R2=', R2)


melhor alpha= 0.1
R2= 0.1382989815818182


In [22]:
# fazendo a regressao lasso transformando a renda em polinomio
y_train_pol = y_train + np.power(y_train, 2)
y_test_pol = y_test + np.power(y_test, 2)

In [23]:
lasso_cv = LassoCV(alphas=alpha_v, cv=5)
lasso_cv.fit(X_train, y_train_pol)

b_alpha = ridge_cv.alpha_
print('melhor alpha=', b_alpha)

y_pred = lasso_cv.predict(X_test)
R2 = r2_score(y_test_pol, y_pred)
print('R2=', R2)

melhor alpha= 0.1
R2= 0.004755974993183698


In [24]:
# com as transformacoes na variavel y nao consegui melhor o modelo

In [25]:
#######7
from sklearn.tree import DecisionTreeRegressor
dtr = DecisionTreeRegressor()
dtr.fit(X_train, y_train)

DecisionTreeRegressor()

In [26]:
y_pred = dtr.predict(X_test)

In [27]:
R2 = r2_score(y_test, y_pred)
print('R2=', R2)

R2= 0.07973023878638252
